In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
from IPython import display

# 語意即向量 (pytorch)

支援python 版本: 3.5以上  
支援pytorch版本 : 1.2以上

在深度學習中，所有計算的基礎都基於向量(張量)，而所謂複雜的問題，往往都是高維空間的計算問題。解決這些高維框間問題的最好辦法就是施以降維攻擊。文字從原本稀疏的onehot編碼，透過基於上下文猜中間的詞(cbow)的簡單任務，就能夠將文字的對應關係與語意從原本數百萬維(詞彙數)的複雜度，濃縮在200~300維之間，變成緊湊的語意特徵向量，這就是詞向量。飽含著濃縮的語意，因為維度被大幅降維，因此複雜的事務也會變簡單，簡單到只需要加減乘除外加上cosine similarity的計算就能解決的。這個實作是我們自然語言系列實做的第一個，我們就來看看這些語言特徵向量能夠理解語意到甚麼程度。

![Alt text](../images/word2vec.jpg)

In [2]:
import os
os.environ['TRIDENT_BACKEND'] = 'pytorch'

#!pip uninstall tridentx
#!pip install tridentx --upgrade
from trident import *
from typing import Optional,List,Tuple
import locale
import datetime
import tqdm

trident 0.6.0


Using Pytorch backend.
Image Data Format: channels_first.
Image Channel Order: rgb.
Using pillow image backend.


Pillow version:7.1.2.


Pytorch version:1.6.0+cu101.
Automatic Mixed Precision Support:True.


為了方便各位理解裡面每個計算的過程，以及順便讓各位看到如何在trident中擴充新的神經層，我把詞向量層的代碼附上。請注意，我這個詞向量當初是使用簡體語料訓練出來的，為了方便繁體中文使用者方便，我在裡面實現了基礎的繁簡轉換，但是不算完美，所以還請包涵(如果有人有足夠的繁體中文語料也請與我聯繫歐)。

In [3]:

class Word2Vec(Embedding):
    """中文詞向量
        繼承Embedding Layer

    """
    def __init__(self, pretrained=False,locale=None, embedding_dim: Optional[int]= None, num_embeddings: Optional[int] = None, vocabs: Optional[List[str]] = None, padding_idx: Optional[int] = None,
                 max_norm: Optional[float] = None, norm_type: float = 2., scale_grad_by_freq: bool = False,
                 sparse: bool = False, _weight: Optional[Tensor] = None, filter_index=-1, keep_output: bool = False, name: Optional[str] = None) -> None:

        """
        Py Word2vec结构
        """
        super().__init__(num_embeddings=num_embeddings,embedding_dim=embedding_dim,max_norm=max_norm,norm_type=norm_type,scale_grad_by_freq=scale_grad_by_freq,sparse=sparse,_weight=_weight,filter_index=filter_index, keep_output= keep_output, name=name)
        self.locale=_locale
        if _locale is None:
            self.locale = locale.getdefaultlocale()[0].lower()
        print('locale:',self.locale)

        self._vocabs = OrderedDict()
        if vocabs is not None:
            for k in range(len(vocabs)):
                self._vocabs[vocabs[k]] = k

    @property
    def vocabs(self):
       #詞彙表
       return self._vocabs


    def word2idx(self, word: str):
        # 文字轉索引(根據locale處理繁簡轉換)
        if self.locale != 'zh_cn' and word in self.tw2cn:
            word=self.tw2cn[word]
        if word in  self._vocabs :
            return self._vocabs[word]
        else:
            return None

    def idx2word(self, index: int):
        # 索引轉文字(根據locale處理繁簡轉換)
        if index < len(self._vocabs):
            word= self._vocabs.key_list[index]
            if self.locale != 'zh_cn'and word in self.cn2tw:
                word=self.cn2tw[word]
            return word
        else:
            return None

    @classmethod
    def load(cls):
        #從google drive載入模型
        st = datetime.datetime.now()

        dirname = os.path.join(get_trident_dir(), 'models')
        download_model_from_google_drive('13XZPWh8QhEsC8EdIp1niLtZz0ipatSGC', dirname, 'word2vec_chinese.pth')
        recovery_model = load(os.path.join(dirname, 'word2vec_chinese.pth'))
        recovery_model.eval()
        recovery_model.locale = locale.getdefaultlocale()[0].lower()
        recovery_model.to(get_device())
        
        if not hasattr(recovery_model, 'tw2cn') or recovery_model.tw2cn is None:
            with open('vocabs_tw.txt', 'r', encoding='utf-8-sig') as f:
                vocabs_tw = f.readlines()
                vocabs_tw = [s.replace('\n', '') for s in vocabs_tw if s != '\n']
                recovery_model.tw2cn = OrderedDict()
                recovery_model.cn2tw = OrderedDict()

                for i, (w, w_cn) in tqdm(enumerate(zip(vocabs_tw, recovery_model._vocabs.keys()))):
                    if w not in recovery_model.tw2cn:
                        recovery_model.tw2cn[w] = w_cn
                    recovery_model.cn2tw[w_cn] = w

        et = datetime.datetime.now()
        print('total loading time:{0}'.format(et - st))
        return recovery_model

    def find_similar(self,reprt:(str,Tensor), n:int=10,ignore_indexes=None):
        #根據文字或是向量查詢空間中最近文字
        reprt_idx=None
        if ignore_indexes is None:
            ignore_indexes=[]
        if isinstance(reprt,str):
            reprt_idx=self.word2idx(reprt)
            ignore_indexes.append(reprt_idx)
            reprt = self.weight[reprt_idx].expand_dims(0) if reprt in self._vocabs else None
        if is_tensor(reprt):
            correlate=element_cosine_distance(reprt,self.weight)[0]
            sorted_idxes=argsort(correlate,descending=True)

            sorted_idxes=sorted_idxes[:n+len(ignore_indexes)]
            
            sorted_idxes=to_tensor([idx for idx in sorted_idxes if  idx.item()  not in ignore_indexes]).long()
            probs=to_list(correlate[sorted_idxes])[:n]
            words=[self.idx2word(idx.item())for idx in sorted_idxes][:n]
            return OrderedDict(zip(words,probs))
        else:
            raise ValueError('Valid reprt should be a word or a tensor .')

    def analogy(self,reprt1:(str,Tensor,list),reprt2:(str,Tensor,list),reprt3:(str,Tensor,list),n:int=10):
        #類比關係 (男人之於女人等於國王之於皇后)
        reprt1_idx=None
        reprt2_idx=None
        reprt3_idx=None
        reprt1_arr = None
        reprt2_arr= None
        reprt3_arr = None
        exclude_list=[]
        if isinstance(reprt1,str):
            reprt1_idx=self.word2idx(reprt1)
            exclude_list.append(reprt1_idx)
            reprt1_arr=self.weight[reprt1_idx].expand_dims(0) if reprt1_idx is not None else None
        elif isinstance(reprt1,Tensor):
            reprt1_arr = reprt1
        elif isinstance(reprt1,list):
            if isinstance(reprt1[0],str):
                reprt1_arr=self.get_words_centroid(*reprt1)
                for item in reprt1:
                    exclude_list.append(self.word2idx(item))

        if isinstance(reprt2, str):
            reprt2_idx = self.word2idx(reprt2)
            exclude_list.append(reprt2_idx)
            reprt2_arr = self.weight[reprt2_idx].expand_dims(0) if reprt2_idx is not None else None
        elif isinstance(reprt2, Tensor):
            reprt2_arr = reprt2
        elif isinstance(reprt2, list):
            if isinstance(reprt2[0], str):
                reprt2_arr = self.get_words_centroid(*reprt2)
                for item in reprt2:
                    exclude_list.append(self.word2idx(item))

        if isinstance(reprt3,str):
            reprt3_idx=self.word2idx(reprt3)
            exclude_list.append(reprt3_idx)
            reprt3_arr=self.weight[reprt3_idx].expand_dims(0) if reprt3_idx is not None else None
        elif isinstance(reprt3,Tensor):
            reprt3_arr = reprt3
        elif isinstance(reprt3,list):
            if isinstance(reprt3[0],str):
                reprt3_arr=self.get_words_centroid(*reprt3)
                for item in reprt3:
                    exclude_list.append(self.word2idx(item))

        if reprt1_arr is not None and reprt2_arr is not None and reprt3_arr is not None:
            reprt4=reprt2_arr-reprt1_arr+reprt3_arr
            return self.find_similar(reprt4,n=n,ignore_indexes=exclude_list)
        else:
            not_find=[]
            if reprt1_arr is None:
                not_find.append(reprt1)
            if reprt2_arr is None:
                not_find.append(reprt2)
            if reprt3_arr is None:
                not_find.append(reprt3)
            raise ValueError(' ,'.join(not_find)+' was not in vocabs.')
        
    def get_words_centroid(self,*args):
        #取得數個文字的向量均值
        centroid=0
        for arg in args:
            reprt_idx=self.word2idx(arg)
            if reprt_idx is not None:
                centroid+=self.weight[reprt_idx].expand_dims(0) if reprt_idx is not None else None
        return centroid/len(args)

    def get_words_vector(self, word):
        #取得單一文字的向量
        reprt_idx=self.word2idx(word)
        if reprt_idx is not None:
            return self.weight[reprt_idx].expand_dims(0) if reprt_idx is not None else None
        return None

    def get_enumerators(self, *args,negative_case=None,n=10,exclude_samples=True):
        #取得整體距離輸入案例最接近，但是離負案例最遠(negative_case)的文字列表
        positive_correlate=0
        negative_correlate=0
        exclude_list=[]
        for arg in args:
            positive_correlate +=element_cosine_distance(self.get_words_vector(arg), self.weight)[0]

        correlate=positive_correlate
        if negative_case is None:
            pass
        else:
            if isinstance(negative_case,str):
                negative_case=[negative_case]
            if isinstance(negative_case,(list,tuple)):
                for arg in negative_case:
                    negative_correlate += element_cosine_distance(self.get_words_vector(arg), self.weight)[0]
                correlate=positive_correlate-negative_correlate
        sorted_idxes = argsort(correlate, descending=True)
        sorted_idxes = sorted_idxes[:n + len(exclude_list)]
        sorted_idxes = to_tensor([idx for idx in sorted_idxes if idx.item() not in exclude_list]).long()
        probs = to_list(correlate[sorted_idxes])[:n]
        words = [self.idx2word(idx.item()) for idx in sorted_idxes][:n]
        return OrderedDict(zip(words, probs))


In [4]:
w2v=Word2Vec.load()

model file is already existing, donnot need download again.


C:\Anaconda3\envs\azuremlenv\lib\site-packages\torch\serialization.py:615: UserWarning: Couldn't retrieve source code for container of type Word2Vec. It won't be checked for correctness upon loading.
  "type " + container_type.__name__ + ". It won't be checked "


total loading time:0:00:07.759720


## 類比關係

首先我們當然是要來重現一下那個經典句子。

In [5]:
w2v.analogy('男人','國王','女人')

{ 王后: 0.6711670160293579, 路易十四: 0.608500599861145, 國王隊: 0.6028344035148621, 薩克拉門托: 0.6001420021057129, 爵士: 0.5992196202278137, 路易十五: 0.5940967798233032, 教皇: 0.5927847027778625, 拿破侖: 0.5908872485160828, 萊恩: 0.5896217226982117, 喬治: 0.5850805640220642 }

我們也要來試一下，被認為是深度學習偏見的經典案例的那一句。

![Alt text](../images/bias.png)

In [6]:
w2v.analogy('男人','工程師','女人')

{ 技術主管: 0.6507548689842224, 技術人員: 0.6456468105316162, 高級工程師: 0.644733190536499, 設計員: 0.6362589597702026, 電氣工程師: 0.6353718638420105, 程序員: 0.6332074999809265, 技術員: 0.5936263203620911, 軟件開發: 0.589859127998352, 項目經理: 0.5882404446601868, 研發部門: 0.585453450679779 }

我們再試試其他的類比關係....

In [7]:
w2v.analogy('張惠妹','阿妹','周杰倫')

{ 周董: 0.8443139791488647, 杰倫: 0.7464918494224548, 哈林: 0.6755087971687317, 昆凌: 0.6727409362792969, 王力宏: 0.6535261869430542, 劉畊宏: 0.6101048588752747, 那姐: 0.5902318954467773, 吳宗憲: 0.5883661508560181, 林俊杰: 0.5817347764968872, 方文山: 0.5801873803138733 }

In [8]:
w2v.analogy('雙子座',['花心','聰明'],'金牛座')

{ 厚道: 0.6142151355743408, 精明: 0.5781048536300659, 小氣: 0.5588235855102539, 福睿斯: 0.5313888192176819, 老實: 0.5267444252967834, 摳門: 0.5054975748062134, 專一: 0.48833465576171875, 福特: 0.48730799555778503, 蒙迪歐: 0.4865236282348633, 翼虎: 0.4837642312049866 }

In [9]:
w2v.analogy('黃曉明','angelababy','胡歌')

{ 霍建華: 0.7969950437545776, 趙麗穎: 0.7508330345153809, 王凱: 0.745437741279602, 江疏影: 0.7421894669532776, 靳東: 0.7030095458030701, 唐嫣: 0.6812393069267273, 黃軒: 0.6749378442764282, 馬思純: 0.6685833930969238, 王子文: 0.6607145667076111, 楊洋: 0.6537554860115051 }

透過文字特徵的抽取，機器也能理解一些專業術語以及抽象關係

In [10]:
w2v.analogy('基金','贖回','期貨')

{ 平倉: 0.5166105031967163, 套保: 0.5061496496200562, 交割: 0.4786132276058197, 期貨市場: 0.47824668884277344, 期貨交易: 0.4673691391944885, 期市: 0.462535560131073, 限倉: 0.45793449878692627, 期貨價格: 0.45300784707069397, 現貨: 0.45138245820999146, 交收: 0.4483429789543152 }

In [11]:
w2v.analogy('鹵肉飯','米飯','牛肉面')

{ 面條: 0.8094525933265686, 饅頭: 0.7205348014831543, 白米飯: 0.7196975350379944, 湯面: 0.7152272462844849, 稀飯: 0.6803690195083618, 白飯: 0.6797654032707214, 面湯: 0.6796507835388184, 炒面: 0.6778760552406311, 玉米面: 0.6749952435493469, 清湯: 0.6691955924034119 }

## 列舉類似詞

列舉類似詞的原理很簡單，就是給幾個案例，然後我們去找尋跟這堆案例的距離總和最近的(cosine距離加總最大)。我們首先來看以下的案例。

In [12]:
w2v.get_enumerators('波蘭','捷克',n=20)

{ 捷克: 1.8335120677947998, 波蘭: 1.8335120677947998, 匈牙利: 1.6807706356048584, 羅馬尼亞: 1.6704165935516357, 奧地利: 1.6387449502944946, 塞爾維亞: 1.601003885269165, 格魯吉亞: 1.5848050117492676, 斯洛伐克: 1.5822561979293823, 白俄羅斯: 1.5615065097808838, 烏克蘭: 1.5612667798995972, 保加利亞: 1.5587888956069946, 斯洛文尼亞: 1.5476022958755493, 克羅地亞: 1.5182491540908813, 拉脫維亞: 1.5003275871276855, 比利時: 1.4710286855697632, 愛沙尼亞: 1.4648040533065796, 立陶宛: 1.463444471359253, 波黑: 1.4622794389724731, 亞美尼亞: 1.4334149360656738, 葡萄牙: 1.4271327257156372 }

In [13]:
w2v.get_enumerators('美金','人民幣',n=20)

{ 人民幣: 1.5656771659851074, 美金: 1.5656771659851074, 美元: 1.5130161046981812, 韓元: 1.2852433919906616, 歐元: 1.2739934921264648, 日元: 1.2584362030029297, 英鎊: 1.2556638717651367, 盧比: 1.2380576133728027, 新台幣: 1.2200360298156738, 比索: 1.1863689422607422, 台幣: 1.1631474494934082, 韓幣: 1.157507061958313, 英磅: 1.1547523736953735, 澳幣: 1.1409333944320679, 盧布: 1.1278762817382812, 港元: 1.1212557554244995, 澳元: 1.0988752841949463, 馬幣: 1.0820610523223877, 美圓: 1.078179121017456, 新幣: 1.063284158706665 }

但是語意就是那麼複雜的東西，我們看看下面案例就知道複雜在哪了...

In [14]:
w2v.get_enumerators('蘋果','香蕉',n=20)

{ 蘋果: 1.2803409099578857, 香蕉: 1.2803409099578857, 橙子: 1.0296084880828857, 蘋果公司: 0.9841457009315491, 菠蘿: 0.9632220268249512, apples: 0.9545717239379883, 梨子: 0.948722243309021, 橘子: 0.9243385791778564, 柚子: 0.9211810827255249, 水梨: 0.9146208763122559, 西紅柿: 0.908187210559845, applewatch: 0.8966492414474487, 蘋果泥: 0.8853136301040649, 美國蘋果公司: 0.8837924599647522, 火龍果: 0.883455753326416, 黃瓜: 0.883190393447876, 庫克: 0.8825982809066772, 黑莓: 0.8763664364814758, 小米: 0.8750108480453491, 鳳梨: 0.8745032548904419 }

那要如何排除蘋果作為科技公司的語意呢?簡單，就只要再減掉一個微軟就可以了。

In [15]:
w2v.get_enumerators('蘋果','香蕉',negative_case='微軟',n=20)

{ 香蕉: 1.2249088287353516, 菠蘿: 1.0341726541519165, 橙子: 1.0271517038345337, 鳳梨: 0.9487094283103943, 黃瓜: 0.9268299341201782, 火龍果: 0.9219046831130981, 橘子: 0.9158288836479187, 水果: 0.9154824614524841, 草莓: 0.9110262989997864, 柚子: 0.9019840359687805, 西紅柿: 0.8930652737617493, 圣女果: 0.8906663656234741, 百香果: 0.8866918087005615, 葡萄干: 0.8833821415901184, 雪糕: 0.8804001808166504, 西瓜: 0.8787985444068909, 梨子: 0.8760403394699097, 櫻桃: 0.8735409379005432, 哈密瓜: 0.8722341656684875, 榴蓮: 0.8707475066184998 }

接著我們就可以把剛才的列舉值與類比關係整合再一起，我們看看會得到甚麼??

In [16]:
resuts=w2v.get_enumerators('美國','中國','泰國','德國',n=10,exclude_samples='全球')
for key in resuts.key_list:
    print(key, w2v.analogy(['國家','澳大利亞'], ['首都','坎培拉'], key, n=1))

美國 { 華盛頓: 0.6603410243988037 }
德國 { 慕尼黑: 0.6870025396347046 }
中國 { 北京: 0.5113794803619385 }
日本 { 東京: 0.6887736320495605 }
澳大利亞 { 悉尼: 0.6312180161476135 }
印度 { 新德里: 0.6747971177101135 }
法國 { 巴黎: 0.7311732769012451 }
歐洲 { 巴黎: 0.5739215016365051 }
南非 { 開普敦: 0.6316591501235962 }
俄羅斯 { 莫斯科: 0.7113748788833618 }


In [17]:
resuts=w2v.get_enumerators('小米','富士康','格力','百度','企業',negative_case='代工廠',n=10)
for key in resuts.key_list:
    print(key, w2v.analogy('騰訊', '馬化騰', key, n=1))


小米 { 雷軍: 0.5789117813110352 }
格力 { 董明珠: 0.523431658744812 }
華為 { 任正非: 0.5416291356086731 }
百度 { 李彥宏: 0.6010352969169617 }
京東 { 阿里巴巴: 0.4972849488258362 }
阿里巴巴 { 馬云: 0.5887972116470337 }
雷軍 { 柳傳志: 0.5596833825111389 }
富士康 { 郭台銘: 0.4854552745819092 }
阿里 { 馬云: 0.5610129833221436 }
bat { 李彥宏: 0.5235958695411682 }


我們也來試試剛才用過星座類比，各位可以觀察一下，使用單純的類比關係跑出來的結果，和我們加上更具體的文字暗示，告訴它我們要找的是性格方面的詞彙，跑出來的結果是有許多的不同。

In [18]:
stars = ['白羊座', '金牛座','雙子座', '巨蟹座', '獅子座', '處女座', '天秤座', '天蝎座', '射手座', '摩羯座', '水瓶座',
             '雙魚座']
for key in stars:
    print(key, w2v.analogy(['雙子座'],['花心'], key, n=5))

白羊座 { 專情: 0.6783909201622009, 專一: 0.6752706170082092, 處女: 0.6722710132598877, 天蝎女: 0.669556736946106, 大男子主義: 0.6648526787757874 }
金牛座 { 小氣: 0.5419931411743164, 專情: 0.5335520505905151, 大男子主義: 0.5258557200431824, 專一: 0.5238891839981079, 厚道: 0.5147523880004883 }
雙子座 { 專情: 0.7229915261268616, 好色: 0.7180087566375732, 處女: 0.6991274356842041, 變心: 0.6933404207229614, 專一: 0.6901572942733765 }
巨蟹座 { 處女: 0.6970768570899963, 專情: 0.6927146315574646, 專一: 0.6858142018318176, 重感情: 0.6763896942138672, 巨蟹: 0.6725311875343323 }
獅子座 { 處女: 0.6827950477600098, 大男子主義: 0.6801401376724243, 天蝎女: 0.6745721101760864, 專情: 0.6715160608291626, 小心眼: 0.653331995010376 }
處女座 { 處女: 0.7114487290382385, 潔癖: 0.6762192249298096, 專情: 0.6519758701324463, 專一: 0.6362700462341309, 好男人: 0.6187808513641357 }
天秤座 { 處女: 0.71076899766922, 專情: 0.680675208568573, 金牛女: 0.6702104210853577, 專一: 0.6638287901878357, 天蝎女: 0.6528943777084351 }
天蝎座 { 專情: 0.7056789398193359, 處女: 0.7056739330291748, 天蝎: 0.7023247480392456, 天蝎女: 0.699754357337951

In [19]:
stars = ['白羊座', '金牛座','雙子座', '巨蟹座', '獅子座', '處女座', '天秤座', '天蝎座', '射手座', '摩羯座', '水瓶座',
             '雙魚座']
for key in stars:
    print(key, w2v.analogy(['雙子座','星座'],['花心','聰明','喜新厭舊','性格'], key, n=5))

白羊座 { 爭強好勝: 0.7158233523368835, 好強: 0.6983240842819214, 不拘小節: 0.6919166445732117, 直率: 0.6615375876426697, 好勝: 0.6610141396522522 }
金牛座 { 厚道: 0.5623552203178406, 穩重: 0.5453934073448181, 精明: 0.5224258303642273, 沉穩: 0.5054288506507874, 小氣: 0.4996333420276642 }
雙子座 { 善變: 0.6851727962493896, 爭強好勝: 0.6708043217658997, 性情: 0.6469757556915283, 固執: 0.6389720439910889, 不拘小節: 0.6365770101547241 }
巨蟹座 { 重感情: 0.6955630779266357, 溫柔體貼: 0.6886541843414307, 善解人意: 0.6844980120658875, 心地善良: 0.6489030122756958, 爭強好勝: 0.6435976028442383 }
獅子座 { 爭強好勝: 0.7128241062164307, 好強: 0.7042688727378845, 懦弱: 0.6535314321517944, 不拘小節: 0.6532723903656006, 好勝: 0.6438639163970947 }
處女座 { 潔癖: 0.660512387752533, 吹毛求疵: 0.6296411156654358, 偏執: 0.6286720633506775, 完美主義: 0.6265790462493896, 不拘小節: 0.6207302212715149 }
天秤座 { 爭強好勝: 0.6361836791038513, 善解人意: 0.6306613087654114, 優柔寡斷: 0.6280789971351624, 隨和: 0.6257696151733398, 不拘小節: 0.6235988140106201 }
天蝎座 { 爭強好勝: 0.6583436131477356, 固執: 0.6442285180091858, 懦弱: 0.639217436313629